In [4]:
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
video_file = "person15_walking_d1_uncomp (1).avi"
clip1 = VideoFileClip(video_file)

In [9]:
clip1.ipython_display(width = 400)

100%|█████████████████████████████████████████████████████████████████████████████▉| 741/742 [00:00<00:00, 1366.21it/s]


In [10]:
cap = cv2.VideoCapture(video_file)

In [13]:
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(frameCount)
print(cap.read())

741
(True, array([[[241, 241, 241],
        [249, 249, 249],
        [250, 250, 250],
        ..., 
        [241, 241, 241],
        [239, 239, 239],
        [243, 243, 243]],

       [[237, 237, 237],
        [228, 228, 228],
        [232, 232, 232],
        ..., 
        [239, 239, 239],
        [235, 235, 235],
        [218, 218, 218]],

       [[224, 224, 224],
        [222, 222, 222],
        [220, 220, 220],
        ..., 
        [235, 235, 235],
        [232, 232, 232],
        [225, 225, 225]],

       ..., 
       [[200, 200, 200],
        [195, 195, 195],
        [197, 197, 197],
        ..., 
        [194, 194, 194],
        [210, 210, 210],
        [193, 193, 193]],

       [[197, 197, 197],
        [187, 187, 187],
        [192, 192, 192],
        ..., 
        [193, 193, 193],
        [201, 201, 201],
        [174, 174, 174]],

       [[186, 186, 186],
        [192, 192, 192],
        [193, 193, 193],
        ..., 
        [181, 181, 181],
        [185, 185, 185],
       